#** Machine learning IB-2024** #** Home task 1.** #Regression, KNN, LinearRegression.

In this homework we will build models to predict the price of an apartment in Russia. below is a description of some columns of the data set. date - date of publication price - price in rubles level- floor, on which is the apartment levels - number of floors in the apartment rooms - number of rooms in the apartment. If value -1, then the apartment is considered apartments. area - area of the apartment. kitchen_area - area of the kitchen. geo_lat - Latitude geo_lon - Longitude building_type - building material. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

Part 0 – Start of work

To start working with the data, we import the libraries that will be needed in this task.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Download the folium library to display the data on the map by coordinates.

In [ ]:
!pip install folium


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Download the data from the CSV file into the data frame.

In [ ]:
df = pd.read_csv('input_data.csv', sep=';')
print(df.columns)

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id'],
      dtype='object')


Show the coordinates of our buildings on the map.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# List of points with width and length
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Adding points to the map
for point in zip(lats, longs): folium.Marker(location=[point[0], point[1]]).add_to(m)

print(df.columns)

display(m)

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id'],
      dtype='object')


Part 1: Preparing data for processing by machine learning models.

**0.5 Ball**. The geography of our observations in the data set is extremely large. However, we know that the cost of apartments in Moscow and St. Petersburg is much higher than the average in Russia. Let's make signs that show whether the apartment is 20 kilometers from the centre of Moscow or whether the apartment is 20 kilometers from the centre of St. Petersburg. Create two signs is_Moscow and is_Saint_Peterburg. To find the distance by the coordinates use the function haversine_distance.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    a = np.sin(d_lat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(d_lon/2)**2
    return 2 * r * np.arcsin(np.sqrt(a))

MOSCOW = (55.558741, 37.378847)
SPB    = (59.57, 30.19)

df["dist_to_msk"] = haversine_distance(df["geo_lat"], df["geo_lon"], MOSCOW[0], MOSCOW[1])
df["dist_to_spb"] = haversine_distance(df["geo_lat"], df["geo_lon"], SPB[0], SPB[1])

df["is_Moscow"] = df["dist_to_msk"] < df["dist_to_spb"]
df["is_Saint_Peterburg"] = df["dist_to_spb"] < df["dist_to_msk"]

df["in_20km_msk"] = df["dist_to_msk"] <= 20
df["in_20km_spb"] = df["dist_to_spb"] <= 20

print(df.columns)

display(df[["geo_lat","geo_lon","dist_to_msk","dist_to_spb",
            "is_Moscow","is_Saint_Peterburg","in_20km_msk","in_20km_spb"]].head())
print("Distance of 20 km:", df[["in_20km_msk","in_20km_spb"]].sum().to_dict())


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id', 'dist_to_msk', 'dist_to_spb',
       'is_Moscow', 'is_Saint_Peterburg', 'in_20km_msk', 'in_20km_spb'],
      dtype='object')


,geo_lat,geo_lon,dist_to_msk,dist_to_spb,is_Moscow,is_Saint_Peterburg,in_20km_msk,in_20km_spb
0,56.780112,60.699355,1442.985747,1799.005464,True,False,False,False
1,44.608154,40.138381,1233.210598,1792.601884,True,False,False,False
2,55.540060,37.725112,21.879751,633.994173,True,False,False,False
3,44.608154,40.138381,1233.210598,1792.601884,True,False,False,False
4,44.738685,37.713668,1203.368149,1724.573016,True,False,False,False


Сводка по 20 км: {'in_20km_msk': 406605, 'in_20km_spb': 21343}


**0.5 Ball**. In our data set there are signs that we can theoretically use, such as postal_code, but we will do this within the framework of homework very-very long. Therefore it is proposed to remove unnecessary signs from the data frame. We will remove geo_lat, geo_lon, object_type, postal_code, street_id, id_region, house_id.

In [ ]:
to_drop = ["geo_lat","geo_lon","object_type","postal_code","street_id","id_region","house_id"]
df = df.drop(columns=[c for c in to_drop if c in df.columns])
df.head()

print(df.columns)


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'building_type', 'dist_to_msk', 'dist_to_spb', 'is_Moscow',
       'is_Saint_Peterburg', 'in_20km_msk', 'in_20km_spb'],
      dtype='object')


What columns are categorical? What numbers are? Categorical: (Your answer) Numbers: (Your answer) Let us encode categorical characters using OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder

target = "price"

cat_cols = ["building_type", "is_Moscow", "is_Saint_Peterburg"]
print(df.columns)
num_cols = ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_msk', 'dist_to_spb', 'in_20km_msk', 'in_20km_spb']

print("The categories:", cat_cols)
print("The number:", num_cols, f"All of it {len(num_cols)}")

if cat_cols:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_cat = pd.DataFrame(
        ohe.fit_transform(df[cat_cols]),
        columns=ohe.get_feature_names_out(cat_cols),
        index=df.index
    )
    df = pd.concat([df.drop(columns=cat_cols), X_cat], axis=1)

df.head()


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'building_type', 'dist_to_msk', 'dist_to_spb', 'is_Moscow',
       'is_Saint_Peterburg', 'in_20km_msk', 'in_20km_spb'],
      dtype='object')
Категориальные: ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
Числовые: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_msk', 'dist_to_spb', 'in_20km_msk', 'in_20km_spb'] всего 11


,date,price,level,levels,rooms,area,kitchen_area,dist_to_msk,dist_to_spb,in_20km_msk,...,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Peterburg_False,is_Saint_Peterburg_True
0,2021-01-01,2451300,15,31,1,30.3,0.0,1442.985747,1799.005464,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1233.210598,1792.601884,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,21.879751,633.994173,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1233.210598,1792.601884,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,1203.368149,1724.573016,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


**0.5 Ball**. We work with numerical characters: 1. Add two characters to your dataset: the number of days from the day of the first observation (the difference between the dates of the ads). Maybe for the price forecast the floor is not as important as the ratio of the floor to the number of floors in the house, add this character. After adding a new character the date column can be removed. 2. The numerical characters may have different orders. Let us compare the numerical characters using StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

In [ ]:
from sklearn.preprocessing import StandardScaler

if "date" in df.columns and np.issubdtype(df["date"].dtype, np.datetime64):
    dmin, dmax = df["date"].min(), df["date"].max()
    df["days_since_start"] = (df["date"] - dmin).dt.days
    df["days_to_end"]      = (dmax - df["date"]).dt.days
    df = df.drop(columns=["date"])

target = "price"
num_cols = [c for c in df.select_dtypes(include=["number"]).columns if c != target]

scaler = StandardScaler()
scaled = scaler.fit_transform(df[num_cols].fillna(df[num_cols].median()))
df[[c + "_scaled" for c in num_cols]] = scaled

print("Scale of numbers:", len(num_cols))
df[[*num_cols[:5], *(c+"_scaled" for c in num_cols[:5])]].head()


Скейлено числовых: 18


,level,levels,rooms,area,kitchen_area,level_scaled,levels_scaled,rooms_scaled,area_scaled,kitchen_area_scaled
0,15,31,1,30.3,0.0,1.622769,2.665027,-0.621470,-0.840577,0.082486
1,5,5,1,33.0,6.0,-0.270043,-0.936859,-0.621470,-0.741051,0.267565
2,4,13,3,85.0,12.0,-0.459324,0.171414,1.106234,1.175756,0.452644
3,3,5,3,82.0,9.0,-0.648605,-0.936859,1.106234,1.065171,0.360105
4,2,3,1,30.0,9.0,-0.837886,-1.213927,-0.621470,-0.851636,0.360105


**2 Ball**. Realize the KNNRegressor class, which should make regression by the method to the nearest neighbors.

In [ ]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean', weights='distance', p=2):
        self.k = int(n_neighbors)
        self.metric = metric
        self.weights = weights
        self.p = p
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float64).reshape(-1)
        assert X.shape[0] == y.shape[0]
        self.X_train, self.y_train = X, y
        return self

    def predict(self, X, block_size=1024):
        assert self.X_train is not None, "First fit"
        A = np.asarray(X, dtype=np.float32)
        B = self.X_train
        ytr = self.y_train
        n_test, n_train = A.shape[0], B.shape[0]
        k = max(1, min(self.k, n_train))
        eps = 1e-12 # Not to be divided by 0 by weight.


        B2 = np.sum(B * B, axis=1)[None, :]

        preds = np.empty(n_test, dtype=np.float32)
        for s in range(0, n_test, block_size):
            e = min(s + block_size, n_test)
            Ab = A[s:e]
            A2 = np.sum(Ab * Ab, axis=1)[:, None]

            D = A2 + B2 - 2.0 * (Ab @ B.T)
            np.maximum(D, 0.0, out=D)
            D = np.sqrt(D, dtype=np.float32)

            kk = min(k, D.shape[1]-1) if D.shape[1] > 1 else 1
            idx = np.argpartition(D, kth=kk, axis=1)[:, :k]
            d_knn = np.take_along_axis(D, idx, axis=1)
            y_knn = ytr[idx]

            if self.weights == 'uniform':
                preds[s:e] = y_knn.mean(axis=1)
            elif self.weights == 'distance':
                w = 1.0 / (d_knn + eps)
                zero = d_knn < eps
                pr = (w * y_knn).sum(axis=1) / w.sum(axis=1)
                if np.any(zero.any(axis=1)):
                    m = zero.any(axis=1)
                    pr[m] = (y_knn[m] * zero[m]).sum(axis=1) / zero[m].sum(axis=1)
                preds[s:e] = pr
            else:
                raise ValueError("Weights must be ‘uniform’ or ‘distance’")

        return preds.astype(np.float64)

**3 Balls**. Realize the LinearRegression class that supports gradient leaks SGD, Momentum, AdaGrad. Use gradient to optimize the MSE loss function.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.0,
                 max_iter=1000, momentum=0.9, batch_size=None, random_state=42):
        self.learning_rate = float(learning_rate)
        self.optimization = optimization
        self.epsilon = float(epsilon)
        self.decay_rate = float(decay_rate)
        self.max_iter = int(max_iter)
        self.momentum = float(momentum)
        self.batch_size = batch_size
        self.random_state = random_state
        self.weights = None
        self.bias = None
        self._v_w = None
        self._v_b = None
        self._G_w = None
        self._G_b = None

    def _to_2d(self, X):
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        return X

    def _batches(self, n):
        if self.batch_size is None or self.batch_size >= n:
            yield np.arange(n)
        else:
            rng = np.random.default_rng(self.random_state)
            idx = np.arange(n)
            rng.shuffle(idx)
            for s in range(0, n, self.batch_size):
                yield idx[s:s+self.batch_size]

    def fit(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        n, d = X.shape
        if self.weights is None:
            self.weights = np.zeros(d, dtype=float)
        if self.bias is None:
            self.bias = 0.0
        if self.optimization == 'Momentum':
            self._v_w = np.zeros_like(self.weights)
            self._v_b = 0.0
        if self.optimization == 'AdaGrad':
            self._G_w = np.zeros_like(self.weights)
            self._G_b = 0.0

        prev_loss = np.inf
        lr0 = self.learning_rate

        for t in range(self.max_iter):
            lr_t = lr0 / (1.0 + self.decay_rate * t)
            for idx in self._batches(n):
                Xb = X[idx]; yb = y[idx]; m = Xb.shape[0]
                y_pred = Xb @ self.weights + self.bias
                err = y_pred - yb
                grad_w = (2.0/m) * (Xb.T @ err)
                grad_b = (2.0/m) * err.sum()

                if self.optimization == 'SGD':
                    self.weights -= lr_t * grad_w
                    self.bias    -= lr_t * grad_b
                elif self.optimization == 'Momentum':
                    self._v_w = self.momentum * self._v_w + lr_t * grad_w
                    self._v_b = self.momentum * self._v_b + lr_t * grad_b
                    self.weights -= self._v_w
                    self.bias    -= self._v_b
                elif self.optimization == 'AdaGrad':
                    self._G_w += grad_w**2
                    self._G_b += grad_b**2
                    self.weights -= (lr_t / (np.sqrt(self._G_w) + self.epsilon)) * grad_w
                    self.bias    -= (lr_t / (np.sqrt(self._G_b) + self.epsilon)) * grad_b
                else:
                    raise ValueError("optimization must be 'SGD', 'Momentum', or 'AdaGrad'")

            loss = self.mse(X, y)
            if abs(prev_loss - loss) < self.epsilon:
                break
            prev_loss = loss
        return self


    def predict(self, X):
        X = self._to_2d(X)
        return X @ self.weights + self.bias

    def mse(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        pred = self.predict(X)
        return float(np.mean((pred - y)**2))

    def r2_score(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        pred = self.predict(X)
        ss_res = np.sum((y - pred)**2)
        ss_tot = np.sum((y - y.mean())**2)
        return float(1 - ss_res / (ss_tot + 1e-12))


Part 2: Experiments with machine learning models.

**3 Balls**. Do experiments with the machine learning methods you have written. Separate the training and test samples at 0.8 and 0.2 respectively. Measure the MSE, MAE, RMSE error. Use the KNNRegressor and LinearRegression methods from the sklearn library, compare the quality of your solutions and library.

In [ ]:
import os
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "4")
os.environ.setdefault("MKL_NUM_THREADS", "4")
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "4")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "4")

if 'df' in globals():
    scaled_cols = [c for c in df.columns if c.endswith('_scaled')]
    if scaled_cols:
        df = df.drop(columns=scaled_cols)

    for c in ['rooms','level','levels','building_type']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='integer')

    for c in ['area','kitchen_area','dist_to_msk','dist_to_spb']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='float')

    for c in ['is_Moscow','is_Saint_Peterburg','in_20km_msk','in_20km_spb']:
        if c in df.columns:
            df[c] = df[c].astype('uint8')


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor as SKKNN
from sklearn.linear_model import LinearRegression as SKLR

def to_f32(a):
    return a.toarray().astype(np.float32) if hasattr(a, 'toarray') else a.astype(np.float32)

target = 'price'
cat_cols = ['building_type'] if 'building_type' in df.columns else []
ohe_ready = [c for c in df.columns if c.startswith('building_type_')]
num_candidates = ['rooms','level','levels','area','kitchen_area', 'days_since_start',
                  'days_to_end','dist_to_msk','dist_to_spb']
num_cols = [c for c in num_candidates if c in df.columns]

if not cat_cols and ohe_ready:
    scaler = StandardScaler()
    X_num = scaler.fit_transform(df[num_cols]) if num_cols else np.empty((len(df),0))
    X = np.hstack([to_f32(X_num), df[ohe_ready].to_numpy(np.float32)])
else:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32)
    ct = ColumnTransformer([('cat', ohe, cat_cols), ('num', StandardScaler(), num_cols)])
    X = to_f32(ct.fit_transform(df[cat_cols + num_cols]))

y = df[target].to_numpy(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = to_f32(X_train); X_test = to_f32(X_test)

rng = np.random.default_rng(42)
max_train = min(60000, len(X_train))
max_test  = min(15000,  len(X_test))
tr_idx = rng.choice(len(X_train), size=max_train, replace=False)
te_idx = rng.choice(len(X_test),  size=max_test,  replace=False)
Xtr_s, ytr_s = X_train[tr_idx], y_train[tr_idx]
Xte_s, yte_s = X_test[te_idx],  y_test[te_idx]

knn_my = KNNRegressor(n_neighbors=5, metric='euclidean', weights='distance')
knn_my.fit(Xtr_s, ytr_s)
y_pred_knn_my = knn_my.predict(Xte_s, block_size=512)

lin_my = LinearRegression(learning_rate=0.1, optimization='AdaGrad', max_iter=2000, epsilon=1e-8)
lin_my.fit(X_train, y_train)
y_pred_lin_my = lin_my.predict(X_test)

knn_sk = SKKNN(n_neighbors=5, weights='distance', algorithm='kd_tree', leaf_size=30, n_jobs=-1)
knn_sk.fit(Xtr_s, ytr_s)
y_pred_knn_sk = knn_sk.predict(Xte_s)

lr_sk = SKLR().fit(X_train, y_train)
y_pred_lr_sk = lr_sk.predict(X_test)

def metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = float(np.sqrt(mse))
    r2 = r2_score(y_true, y_pred)
    return mse, mae, rmse, r2

rows = [
    ('KNN',) + metrics(yte_s, y_pred_knn_my),
    ('KNN (sklearn)',) + metrics(yte_s, y_pred_knn_sk),
    ('Linear',) + metrics(y_test, y_pred_lin_my),
    ('Linear (sklearn)',) + metrics(y_test, y_pred_lr_sk),
]
res = pd.DataFrame(rows, columns=['Model','MSE','MAE','RMSE','R2']).sort_values('RMSE')
display(res)
